## 03 - The True Power of Qurry

In previous section, we have learned how to use Qurry in basic way, it's kind of boring and not attractive for anyone, even you may ask why should I use this tool. In this section, we will learn how to use Qurry in advanced way, where the true power of Qurry lies.



In [1]:
from qurry import EntropyMeasure, backendWrapper, backendManager
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)

expProgram02 = EntropyMeasure()

## 3.1 - `backendManager` the advanced version `backendWrapper`

In previous chapter, we used `backendWrapper` to import our backend, and put provider input it to load IBM Backend from our IBM Account. 
Now, `backendManager` will load accout automatically, and you can use `backendManager` to load backend directly.

```python
backendManager.save_account('your_token')
```
To save your account, if you have already saved, you can ignore this step.

```python
backend = backendManager(
    instance='ibm-q/open/main'
)
```
or 
```python
backend = backendManager(
    hub='ibm-q', group='open', project='main'
)
```


In [2]:
backend = backendManager(
    hub='ibm-q', group='open', project='main'
)

| Provider by 'qiskit_ibm_provider'.


You can just type the name of backend wiithout the prefix of `'ibmq_'` or `'ibm_'`.


In [3]:
backend('ibmq_belem'), backend('belem')

(<IBMBackend('ibmq_belem')>, <IBMBackend('ibmq_belem')>)

## 3.2 - Launching a multiJob

Consider a scenario where you have multiple circuit want to run on a backend, and you want to run them at the same time. In this case, you can use `multiOutput` to launch a multiJob.

For our example, we will show how to measure 100 circuits with 100 times randomized measure, 4096 shots, and mesure their entropy with multiple subsystem divisions at the same time.

### 1. Loading 100 circuits

(Topological Paramagnetic State is already the most complicated case in Qurry, so we will use it as our example, ususally I prefer some more complicated circuits like sshxx model with 20+ trotter steps, but it's not necessary for this example.)

```python

In [5]:
from qurry.case import topParamagnet

sample = topParamagnet(8, 'period')
sample.circuit
for i in range(100):
    expProgram02.add(sample.circuit)


In [7]:
expProgram02.waves.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

### 2. Preparing configurations

For launch 100 experiment, it requires 100 configurations for each experiment, since they are similar, we can just make a list.

For randomized measure, each configuration is in this form of dictionary:

```python
{
    'waves': 'the circuit to measure',
    'tags': 'tags for this job',
    'shots': '...',
    'times': 100,
}
```

In [11]:
from typing import TypedDict, Hashable, Iterable, Union, Optional

class randomizedConfig(TypedDict):
    waves: Hashable
    tags: Union[Iterable[Hashable], Hashable]
    times: Optional[int]
    """Default: 100 in :cls:`RandomizedMeasure`"""
    

In [10]:
configList: list[randomizedConfig] = [{
    'wave': i,
    'tags': ('topParamagnet', int(i/10)),
    'times': 100,
} for i in range(100)]

### 3. Launching multiJob

Attenetion, this example may make your computer go brrrr.

In [12]:
from pathlib import Path

hashID = expProgram02.multiOutput(
    configList=configList,
    backend=backend('aer'),
    saveLocation=Path('./'),
    summonerName='example.multiOutput',
    shots=4096,
)

# It takes 10m 41.2s to run this cell on my computer.

| MultiManager building...


| Write "example.multiOutput.qurry.001", at location "example.multiOutput.qurry.001": 
| 100/100 100%|██████████| - Experiments build - 03:39 < 00:00


| Export multimanager...


| 10/10 - exporting done:  - 00:00 < 00:00                      
| 0/0 - exporting quantity - 00:00 < ?


| Export multi.config.json for a2556f65-ef80-4410-9b65-4b1c9409f498


...

| Export multimanager...


| 10/10 - exporting done:  - 00:04 < 00:00                      
| 0/0 - exporting quantity - 00:00 < ?


| Export multi.config.json for a2556f65-ef80-4410-9b65-4b1c9409f498


| 100/100 - Multimanger experiment write in a2556f65-ef80-4410-9b65-4b1c9409f498...done:  - 00:55 < 00:00                               


'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'.../r'example.multiOutput.qurry.001/multi.config.json' exported successfully.
| Compress multimanager of 'example.multiOutput.qurry.001'...done


In [13]:
hashID # The hashID of this multiOutput

'a2556f65-ef80-4410-9b65-4b1c9409f498'

In [14]:
expProgram02.multimanagers

{'a2556f65-ef80-4410-9b65-4b1c9409f498': <qurry.qurrium.multimanager.multimanager.MultiManager at 0x7f4a725ac190>}

### 4. Make multiple analysis with multiple subsystem divisions


In [17]:
subsytems = [
    2, 4, 6, (2, 4), (4, 6), (2, 6), 
    (-2, 2), (-4, 2), (-2, 4), 3, (5, 7), (4, 7)]
# this is a list of subsystems that we want to measure

In [ ]:
for d in subsytems:
    expProgram02.multiAnalysis(
        summonerID=hashID,
        degree=d,
        analysisName=(
            f'N_A={"to".join([str(dd) for dd in d])}' if isinstance(d, tuple) 
            else f'N_A={d}'),
        noSerialize=True,
    )

## 3.3 Reading and Writing Data

In [ ]:
hashID = expProgram02.multiRead(
    saveLocation=Path('./'),
    summonerName='exmaple.multiOutput.qurry.001',
)